In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.linalg import LinAlgError
import scipy.stats as stats
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

In [2]:
from base import reduce_mem_usage, read_csv, datapath, drop_features, tmppath, Box_Cox, train_drop_features, getTrainTest, minmax_target, combine_feature, auto_feature_make

from base import evaluation

In [3]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb

In [4]:
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

In [5]:
train = reduce_mem_usage(read_csv(tmppath + 'Box_Subtrain.csv'))
test = reduce_mem_usage(read_csv(tmppath + 'Box_Subtest.csv'))

Memory usage after optimization is: 3.96 MB
Decreased by 50.8%
Memory usage after optimization is: 1.14 MB
Decreased by 50.0%


In [6]:
X_train = train.drop(['emd_lable2'], axis=1)
Y_train = train['emd_lable2'].astype(int)

In [7]:
discrete_list = ['seg_flight', 'seg_cabin', 'pref_orig_m6_2', 'pref_line_y1_2',
                 'pref_line_y1_3', 'pref_line_y2_2', 'pref_line_y2_3', 'pref_line_y3_3'
    , 'pref_line_y3_4', 'pref_line_y3_5', 'pref_aircraft_y3_3', 'pref_city_y1_2',
                 'pref_city_y3_4', 'pref_dest_city_m6', 'pref_dest_city_y3'
    , 'pref_month_y3_1', 'seg_dep_time_month']  # 训练中需要剔除的特征都是离散型的特征

In [8]:
feature_list = X_train.columns.tolist()
continue_list = list(set(feature_list) - set(discrete_list))

In [9]:
X_train, X_test = minmax_target(X_train, test, Y_train, continue_list, discrete_list) # 离散值编码与连续特征归一化

D:\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [10]:
x_train, x_test, y_train, y_test =getTrainTest(X_train, Y_train)# 线下验证，80%训练集，20%验证集

In [11]:
dtrain = xgb.DMatrix(x_train[continue_list], label=y_train)
dtest = xgb.DMatrix(x_test[continue_list], label=y_test)

In [12]:
xgb1 = XGBClassifier(max_depth=3,
                     learning_rate=0.1,
                     n_estimators=10000,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0,
                     min_child_weight=5,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)

In [13]:
xgb1.get_xgb_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': 'gbtree',
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.8,
 'gamma': 0,
 'gpu_id': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_delta_step': None,
 'max_depth': 3,
 'min_child_weight': 5,
 'monotone_constraints': None,
 'n_jobs': 4,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': 0.8,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'seed': 27}

初始自定义参数下的得分结果


In [14]:
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:29:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.6235640301318268 0.9208733179908042
f1= 0.3556581986143187 0.9098398169336385
precision_score= 0.5579710144927537 0.9890547263681592
recall_score= 0.26101694915254237 0.8423728813559322
accuracy= 0.9404736505227225 0.9894905308082155
auc= 0.6235640301318268 0.9208733179908042
#####混淆矩阵#########
[[4331   61]
 [ 218   77]] [[17554    11]
 [  186   994]]


In [15]:
cv_result = xgb.cv(xgb1.get_xgb_params(),
dtrain,
num_boost_round=xgb1.get_params()['n_estimators'],
nfold=5,
metrics='auc',
early_stopping_rounds=50,
callbacks=[xgb.callback.early_stop(50),
xgb.callback.print_evaluation(period=1,show_stdv=True)])

Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 50 rounds.
[0]	train-auc:0.65991+0.00298	test-auc:0.65879+0.01129
[1]	train-auc:0.66150+0.00346	test-auc:0.65958+0.01008
[2]	train-auc:0.66822+0.00615	test-auc:0.66270+0.01364
[3]	train-auc:0.66921+0.00535	test-auc:0.66616+0.01110
[4]	train-auc:0.67106+0.00484	test-auc:0.66835+0.01233
[5]	train-auc:0.67225+0.00446	test-auc:0.66882+0.01203
[6]	train-auc:0.67511+0.00457	test-auc:0.67103+0.01148
[7]	train-auc:0.67577+0.00389	test-auc:0.67126+0.01081
[8]	train-auc:0.67801+0.00490	test-auc:0.67399+0.01531
[9]	train-auc:0.69049+0.00794	test-auc:0.68302+0.01252
[10]	train-auc:0.69421+0.00720	test-auc:0.68607+0.00950
[11]	train-auc:0.69954+0.00990	test-auc:0.68964+0.00832
[12]	train-auc:0.70463+0.01239	test-auc:0.69371+0.01000
[13]	train-auc:0.70634+0.01213	test-auc:0.69384+0.00862
[14]	train-auc:0.71120+0.01138	test-auc:0.70034+0.01196
[15]	train-auc:0.71699+0.0103

D:\anaconda3\lib\site-packages\xgboost\training.py:20: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


得到最佳迭代次数为496

In [16]:
xgb1 = XGBClassifier(max_depth=3,
                     learning_rate=0.1,
                     n_estimators=496,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0,
                     min_child_weight=5,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)


In [17]:
##修改为最佳迭代次数后的得分结果
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:37:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.5279028125096478 0.5822285988604071
f1= 0.10625 0.28077753779697623
precision_score= 0.68 0.9330143540669856
recall_score= 0.0576271186440678 0.1652542372881356
accuracy= 0.9389801578835075 0.9467057882101894
auc= 0.5279028125096477 0.5822285988604071
#####混淆矩阵#########
[[4384    8]
 [ 278   17]] [[17551    14]
 [  985   195]]


调优max_depth和min_child_weight参数


In [18]:
param_grid = {'max_depth':range(1,9,2),
'min_child_weight':range(1,9,2)}
grid_search = GridSearchCV(xgb1,param_grid,scoring='f1',iid=False,cv=5)
grid_search.fit(x_train,y_train)
print('best_params:', grid_search.best_params_)
print('best_score:', grid_search.best_score_)

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


[11:25:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

得到最佳max_depth和min_child_weight分别为7和1，下面在进行一次范围更小的调优

In [19]:
param_grid = {'max_depth':[6,7,8],
'min_child_weight':[1,2,3]}
grid_search = GridSearchCV(xgb1,param_grid,scoring='f1',iid=False,cv=5)
grid_search.fit(x_train,y_train)
print('best_params:', grid_search.best_params_)
print('best_score:', grid_search.best_score_)

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


[11:32:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:32:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:32:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:32:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

得到最佳max_depth和min_child_weight分别为8和1

In [20]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.1,
                     n_estimators=496,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)

In [21]:
##修改为最佳max_depth和min_child_weight后的得分结果
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:38:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.6090183229909543 0.9153933091133659
f1= 0.34196891191709844 0.9037309995393827
precision_score= 0.7252747252747253 0.9899091826437941
recall_score= 0.22372881355932203 0.8313559322033899
accuracy= 0.9458075528056326 0.9888503600960256
auc= 0.6090183229909543 0.9153933091133659
#####混淆矩阵#########
[[4367   25]
 [ 229   66]] [[17555    10]
 [  199   981]]


下面对gamma参数进行调优

In [22]:
param_grid = {'gamma':[1,2,3,4,5,6,7,8,9]}
grid_search = GridSearchCV(xgb1,param_grid,scoring='f1',iid=False,cv=5)
grid_search.fit(x_train,y_train)
print('best_params:', grid_search.best_params_)
print('best_score:', grid_search.best_score_)

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


[11:46:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:46:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:46:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:47:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

这里发现最优gamma值为1 但是分数反而下降了，所以在精细化一些调优

In [23]:
param_grid = {'gamma':[i/10.0 for i in range(0,10)]}
grid_search = GridSearchCV(xgb1,param_grid,scoring='f1',iid=False,cv=5)
grid_search.fit(x_train,y_train)
print('best_params:', grid_search.best_params_)
print('best_score:', grid_search.best_score_)



D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


[14:01:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:02:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:02:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:02:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

这里得到最优gamma值为0.1，分数有所提高

In [24]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.1,
                     n_estimators=496,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)


In [25]:

##修改为最佳gamma后的得分结果
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:07:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.6138753820505696 0.9154502405110316
f1= 0.35294117647058826 0.9045643153526972
precision_score= 0.71875 0.9919110212335692
recall_score= 0.23389830508474577 0.8313559322033899
accuracy= 0.946020908896949 0.9889570552147239
auc= 0.6138753820505696 0.9154502405110317
#####混淆矩阵#########
[[4365   27]
 [ 226   69]] [[17557     8]
 [  199   981]]


下面调优subsample和colsample_bytree参数

In [26]:
param_grid = {'subsample':[i/10.0 for i in range(5,11)],
              'colsample_bytree':[i/10.0 for i in range(5,11)]}
grid_search = GridSearchCV(xgb1,param_grid,scoring='f1',iid=False,cv=5)
grid_search.fit(x_train,y_train)
print('best_params:', grid_search.best_params_)
print('best_score:', grid_search.best_score_)

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


[14:09:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:09:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:10:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:10:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

得出分数有所提高，最佳colsample_bytree值为0.9，最佳subsample值为0.7

In [27]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.1,
                     n_estimators=496,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.7,
                     colsample_bytree=0.9,
                     seed=27)


In [28]:


##修改为最佳gamma后的得分结果
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:24:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.598621144762434 0.9201112574601842
f1= 0.3141361256544503 0.910091743119266
precision_score= 0.6896551724137931 0.992
recall_score= 0.2033898305084746 0.8406779661016949
accuracy= 0.9441007040751014 0.9895438783675646
auc= 0.5986211447624339 0.9201112574601843
#####混淆矩阵#########
[[4365   27]
 [ 235   60]] [[17557     8]
 [  188   992]]


这里验证的结果发现结果变差了，故仍然采用未改动之前的参数


In [29]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.1,
                     n_estimators=496,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)


调节正则化参数

In [31]:
param_grid = {'reg_lambda':[i/10.0 for i in range(1,11)]}
grid_search = GridSearchCV(xgb1, param_grid, scoring='f1', iid=False, cv=5)
grid_search.fit(x_train, y_train)
print('best_params:', grid_search.best_params_)
print('best_score:', grid_search.best_score_)

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


[14:34:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:34:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:34:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:34:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

得出f1分数有所提高，最优reg_labmda参数为0.3

In [32]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.1,
                     n_estimators=496,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     reg_lambda=0.3,
                     seed=27)


In [33]:


##修改为最佳reg_lambda后的得分结果
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:40:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.6072095643851687 0.9230488934562666
f1= 0.3367875647668394 0.9131627056672761
precision_score= 0.7142857142857143 0.9910714285714286
recall_score= 0.22033898305084745 0.8466101694915255
accuracy= 0.9453808406229998 0.9898639637236596
auc= 0.6072095643851687 0.9230488934562665
#####混淆矩阵#########
[[4366   26]
 [ 230   65]] [[17556     9]
 [  181   999]]


发现修改后测试集分数反而降低，故改回

In [34]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.1,
                     n_estimators=496,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)

最后调低速率，增加迭代次数

In [35]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.05,
                     n_estimators=10000,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)

In [36]:
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:42:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.6483174338550832 0.9897418547091432
f1= 0.4155251141552512 0.9825902335456476
precision_score= 0.6363636363636364 0.9846808510638297
recall_score= 0.30847457627118646 0.9805084745762712
accuracy= 0.9453808406229998 0.9978127500666845
auc= 0.6483174338550832 0.9897418547091432
#####混淆矩阵#########
[[4340   52]
 [ 204   91]] [[17547    18]
 [   23  1157]]


继续降低速率测试

In [37]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.01,
                     n_estimators=10000,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)


In [38]:
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:45:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.6347708468401717 0.9805336836061698
f1= 0.3961352657004831 0.9746672391584371
precision_score= 0.6890756302521008 0.9878154917319408
recall_score= 0.27796610169491526 0.961864406779661
accuracy= 0.9466609771708983 0.9968524939983996
auc= 0.6347708468401717 0.9805336836061698
#####混淆矩阵#########
[[4355   37]
 [ 213   82]] [[17551    14]
 [   45  1135]]


发现继续降低速率效果并不好，故改回速率，然后用cv函数测试最佳迭代次数

In [44]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.05,
                     n_estimators=10000,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)


In [45]:
cv_result = xgb.cv(xgb1.get_xgb_params(),
dtrain,
num_boost_round=xgb1.get_params()['n_estimators'],
nfold=5,
metrics='auc',
early_stopping_rounds=100,
callbacks=[xgb.callback.early_stop(100),
xgb.callback.print_evaluation(period=1,show_stdv=True)])

Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
[0]	train-auc:0.71898+0.01835	test-auc:0.69875+0.01339
[1]	train-auc:0.73548+0.00923	test-auc:0.71058+0.02621
[2]	train-auc:0.74033+0.00971	test-auc:0.71179+0.02656
[3]	train-auc:0.74517+0.00719	test-auc:0.71439+0.02418
[4]	train-auc:0.75047+0.00693	test-auc:0.72100+0.02672
[5]	train-auc:0.75515+0.00930	test-auc:0.72116+0.02829
[6]	train-auc:0.76416+0.00615	test-auc:0.72299+0.02655
[7]	train-auc:0.76819+0.00455	test-auc:0.72564+0.02560
[8]	train-auc:0.77375+0.00698	test-auc:0.72712+0.02542
[9]	train-auc:0.77722+0.00748	test-auc:0.72705+0.02525
[10]	train-auc:0.78077+0.00580	test-auc:0.72767+0.02692
[11]	train-auc:0.78365+0.00748	test-auc:0.72812+0.02596
[12]	train-auc:0.78577+0.00685	test-auc:0.72938+0.02613
[13]	train-auc:0.78965+0.00499	test-auc:0.73226+0.02394
[14]	train-auc:0.79356+0.00366	test-auc:0.73417+0.02503
[15]	train-auc:0.79715+0.004

D:\anaconda3\lib\site-packages\xgboost\training.py:20: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


得到最佳迭代次数为334，修改为334测试得分

In [41]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.05,
                     n_estimators=334,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)

In [42]:
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:50:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.5397672192893087 0.6752529346205619
f1= 0.14678899082568805 0.5175
precision_score= 0.75 0.9857142857142858
recall_score= 0.08135593220338982 0.35084745762711866
accuracy= 0.9404736505227225 0.9588156841824487
auc= 0.5397672192893087 0.6752529346205619
#####混淆矩阵#########
[[4384    8]
 [ 271   24]] [[17559     6]
 [  766   414]]


In [46]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.01,
                     n_estimators=10000,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)

In [47]:
cv_result = xgb.cv(xgb1.get_xgb_params(),
dtrain,
num_boost_round=xgb1.get_params()['n_estimators'],
nfold=5,
metrics='auc',
early_stopping_rounds=100,
callbacks=[xgb.callback.early_stop(100),
xgb.callback.print_evaluation(period=1,show_stdv=True)])

Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
[0]	train-auc:0.71898+0.01835	test-auc:0.69875+0.01339
[1]	train-auc:0.73414+0.00877	test-auc:0.71026+0.02635
[2]	train-auc:0.73687+0.01013	test-auc:0.71095+0.02558
[3]	train-auc:0.74004+0.00829	test-auc:0.71338+0.02348
[4]	train-auc:0.74397+0.00744	test-auc:0.71888+0.02758
[5]	train-auc:0.74595+0.00856	test-auc:0.71984+0.02759
[6]	train-auc:0.74893+0.00579	test-auc:0.72063+0.02613
[7]	train-auc:0.74913+0.00598	test-auc:0.72119+0.02559
[8]	train-auc:0.75060+0.00462	test-auc:0.72172+0.02546
[9]	train-auc:0.75443+0.00449	test-auc:0.72199+0.02384
[10]	train-auc:0.75550+0.00484	test-auc:0.72226+0.02527
[11]	train-auc:0.75641+0.00457	test-auc:0.72220+0.02525
[12]	train-auc:0.75690+0.00361	test-auc:0.72300+0.02491
[13]	train-auc:0.75835+0.00409	test-auc:0.72446+0.02362
[14]	train-auc:0.76013+0.00233	test-auc:0.72422+0.02451
[15]	train-auc:0.76300+0.004

D:\anaconda3\lib\site-packages\xgboost\training.py:20: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


In [48]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.01,
                     n_estimators=1493,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)

In [49]:
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:58:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.5396533759377605 0.6557614091968331
f1= 0.1463414634146341 0.4736164736164737
precision_score= 0.7272727272727273 0.983957219251337
recall_score= 0.08135593220338982 0.31186440677966104
accuracy= 0.940260294431406 0.9563616964523873
auc= 0.5396533759377605 0.6557614091968331
#####混淆矩阵#########
[[4383    9]
 [ 271   24]] [[17559     6]
 [  812   368]]


In [64]:
xgb1 = XGBClassifier(max_depth=8,
                     learning_rate=0.05,
                     n_estimators=6000,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0.1,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     seed=27)

In [65]:
xgb_bst1 = xgb1.fit(x_train, y_train)
y_pred = xgb_bst1.predict(x_test)
y_pred2 = xgb_bst1.predict(x_train)
print("XGBoost结果")
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test),balanced_accuracy_score(y_pred=y_pred2, y_true=y_train))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test), f1_score(y_pred=y_pred2, y_true=y_train))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test),precision_score(y_pred=y_pred2, y_true=y_train))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test), recall_score(y_pred=y_pred2, y_true=y_train))
print("accuracy=", accuracy_score(y_pred=y_pred, y_true=y_test), accuracy_score(y_pred=y_pred2, y_true=y_train))
print("auc=", roc_auc_score(y_true=y_test, y_score=y_pred), roc_auc_score(y_true=y_train, y_score=y_pred2))
print("#####混淆矩阵#########")
print(confusion_matrix(y_true=y_test, y_pred=y_pred), confusion_matrix(y_true=y_train, y_pred=y_pred2))


D:\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:22:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost结果
balanced_accuracy_score= 0.6490004939643728 0.9901371178238696
f1= 0.42129629629629634 0.982605006364022
precision_score= 0.6642335766423357 0.983857264231096
recall_score= 0.30847457627118646 0.9813559322033898
accuracy= 0.9466609771708983 0.9978127500666845
auc= 0.6490004939643728 0.9901371178238696
#####混淆矩阵#########
[[4346   46]
 [ 204   91]] [[17546    19]
 [   22  1158]]
